In [1]:
import bempp.api 
import numpy as np
import mtf

from mtf.utils import bhmie
from mtf.config import config
from mtf.functions import define_bempp_functions

from matplotlib import pyplot as plt
from decimal import Decimal

bempp.api.enable_console_logging()
M = 2

mtf.config.set_case("B")

tangential_trace, neumann_trace = define_bempp_functions(config)

k0, k1 = config["k_ext"], config["k_int"]
lambda_par, freq = config["lambda"], config["frequency"]

polarization = config["polarization"]
direction = config["direction"]

eps_rel = config["eps_rel"]
mu_rel = config["mu_rel"]
mu0 = config["mu_ext"]
mu1 = mu_rel * mu0

eta_rel = np.sqrt(mu_rel / eps_rel)

print("The exterior wavenumber is: {0}".format(k0))
print("The interior wavenumber is: {0}".format(k1))

print("----")
print("The exterior wavelenght is: {0}".format(lambda_par))
print("The exterior frequency is: {:.2E}".format(Decimal(freq)))

The exterior wavenumber is: 5.0
The interior wavenumber is: 6.892024376045111
----
The exterior wavelenght is: 1.2566370614359172
The exterior frequency is: 2.39E+8


In [2]:
segments = [[10], [10]]
swapped_normals = [[10], []]

k_int, k_ext = config["k_int"], config["k_ext"]

n = k_int / k_ext
refIndex = n
numAngles = 901
s1, s2, qext, qsca, qback, gsca = bhmie(k_ext, k_int / k_ext, numAngles)
angles = config['angles']

#transmission_operators = assemble_operators(grid, config)
#far_field, solution = evaluate_far_field(transmission_operators, config)

k_list = [k0]
eta_rel_list = [1]
mu_list = [mu0]

for index in range(M-1):
  k_list.append(k1)
  mu_list.append(mu1)
  eta_rel_list.append(eta_rel)

In [3]:
from bempp.api.assembly.boundary_operator import BoundaryOperator
from bempp.api.operators.boundary.maxwell import osrc_mte

class osrcMtE(BoundaryOperator):
    def __init__(self, wf, domain, range_, dual_to_range, parameters=None):
        self.wf = wf
        self._domain = domain
        self._range = range_
        self._dual_to_range = dual_to_range
        self._parameters = parameters
        
    def weak_form(self):
        return self.wf

def osrc_MtE(domain, range_, dual_to_range, p1d, wave_number, parameters=None):
    mte = bempp.api.operators.boundary.maxwell.osrc_mte( [dual_to_range, p1d],  [dual_to_range, p1d],  [dual_to_range, p1d], wave_number)
    wf = mte._assemble()
    return osrcMtE(wf, domain, domain, dual_to_range, parameters)    

#def osrc_MtE(grid, wave_number):
#    rwg = bempp.api.function_space(grid, 'RWG', 0)
#    snc = bempp.api.function_space(grid, 'SNC', 0)
#    p1d = bempp.api.function_space(grid, 'DP', 1)
#    mte = bempp.api.operators.boundary.maxwell.osrc_mte( [snc,p1d],  [snc, p1d],  [snc, p1d], wave_number)
#    wf = mte._assemble()
#    return osrcMtE(wf, rwg, rwg, snc)    

In [4]:
"""Iterative solver interfaces."""

import numpy as _np
import scipy
# pylint: disable=invalid-name
# pylint: disable=too-many-arguments
# pylint: disable=too-many-locals


class IterationCounter(object):
    """Iteration Counter class."""

    def __init__(self, store_residuals, iteration_is_cg=False, operator=None, rhs=None):
        self._count = 0
        self._store_residuals = store_residuals
        self._residuals = []
        self._iteration_is_cg = iteration_is_cg
        self._operator = operator
        self._rhs = rhs

    def __call__(self, x):
        """Call."""
        from bempp.api import log

        self._count += 1
        if self._store_residuals:
            if self._iteration_is_cg:
                res = self._rhs - self._operator * x
            else:
                res = x
            self._residuals.append(_np.linalg.norm(res))
            log(f"GMRES Iteration {self._count} with residual {self._residuals[-1]}")
        else:
            log(f"GMRES Iteration {self._count}")

    @property
    def count(self):
        """Return the number of iterations."""
        return self._count

    @property
    def residuals(self):
        """Return the vector of residuals."""
        return self._residuals


def gmres(
    A,
    b,
    tol=1e-5,
    restart=None,
    maxiter=None,
    use_strong_form=False,
    return_residuals=False,
    return_iteration_count=False,
):
    """Perform GMRES solve via interface to scipy.

    This function behaves like the scipy.sparse.linalg.gmres function. But
    instead of a linear operator and a vector b it takes a boundary operator
    and a grid function or a blocked operator and a list of grid functions.
    The result is returned as a grid function or as a list of grid functions
    in the correct spaces.

    """
    from bempp.api.assembly.boundary_operator import BoundaryOperator
    from bempp.api.assembly.blocked_operator import BlockedOperatorBase

    if isinstance(A, BoundaryOperator) or isinstance(A, scipy.sparse.linalg.LinearOperator):
        return _gmres_single_op_imp(
            A,
            b,
            tol,
            restart,
            maxiter,
            use_strong_form,
            return_residuals,
            return_iteration_count,
        )

    if isinstance(A, BlockedOperatorBase):
        return _gmres_block_op_imp(
            A,
            b,
            tol,
            restart,
            maxiter,
            use_strong_form,
            return_residuals,
            return_iteration_count,
        )
    

    raise ValueError("A must be a BoundaryOperator or BlockedBoundaryOperator")


def cg(
    A,
    b,
    tol=1e-5,
    maxiter=None,
    use_strong_form=False,
    return_residuals=False,
    return_iteration_count=False,
):
    """Perform CG solve via interface to scipy.

    This function behaves like the scipy.sparse.linalg.cg function. But
    instead of a linear operator and a vector b it takes a boundary operator
    and a grid function. The result is returned as a grid function in the
    correct space.

    """
    from bempp.api.assembly.boundary_operator import BoundaryOperator
    from bempp.api.assembly.grid_function import GridFunction

    import scipy.sparse.linalg

    import bempp.api
    import time

    if not isinstance(A, BoundaryOperator):
        raise ValueError("A must be of type BoundaryOperator")

    if not isinstance(b, GridFunction):
        raise ValueError("b must be of type GridFunction")

    if use_strong_form:
        if not A.range.is_compatible(b.space):
            raise ValueError(
                "The range of A and the domain of A must "
                + "have the same number of unknowns if the strong form is used."
            )
        A_op = A.strong_form()
        b_vec = b.coefficients
    else:
        A_op = A.weak_form()
        b_vec = b.projections(A.dual_to_range)

    callback = IterationCounter(return_residuals, True, A_op, b_vec)
    bempp.api.log("Starting CG iteration")
    start_time = time.time()
    x, info = scipy.sparse.linalg.cg(
        A_op, b_vec, tol=tol, maxiter=maxiter, callback=callback
    )
    end_time = time.time()
    bempp.api.log(
        "CG finished in %i iterations and took %.2E sec."
        % (callback.count, end_time - start_time)
    )

    if isinstance(x, np.ndarray):
        res_fun = x
    else:
        res_fun = GridFunction(A.domain, coefficients=x.ravel())

    if return_residuals and return_iteration_count:
        return res_fun, info, callback.residuals, callback.count

    if return_residuals:
        return res_fun, info, callback.residuals

    if return_iteration_count:
        return res_fun, info, callback.count

    return res_fun, info


def _gmres_single_op_imp(
    A,
    b,
    tol=1e-5,
    restart=None,
    maxiter=None,
    use_strong_form=False,
    return_residuals=False,
    return_iteration_count=False,
):
    """Run implementation of GMRES for single operators."""
    from bempp.api.assembly.grid_function import GridFunction

    import scipy.sparse.linalg

    import bempp.api
    import time

    if not isinstance(b, GridFunction) and not isinstance(b, np.ndarray):
        raise ValueError("b must be of type GridFunction or numpy array")

    # Assemble weak form before the logging messages
    if isinstance(A, scipy.sparse.linalg.LinearOperator):
        A_op = A
        b_vec = b
    elif use_strong_form:
        if not A.range.is_compatible(b.space):
            raise ValueError(
                "The range of A and the domain of A must have"
                + "the same number of unknowns if the strong form is used."
            )
        A_op = A.strong_form()
        b_vec = b.coefficients
    else:
        A_op = A.weak_form()
        b_vec = b.projections(A.dual_to_range)

    callback = IterationCounter(return_residuals)

    bempp.api.log("Starting GMRES iteration")
    start_time = time.time()
    x, info = scipy.sparse.linalg.gmres(
        A_op, b_vec, tol=tol, restart=restart, maxiter=maxiter, callback=callback
    )
    end_time = time.time()
    bempp.api.log(
        "GMRES finished in %i iterations and took %.2E sec."
        % (callback.count, end_time - start_time)
    )

    if isinstance(x, np.ndarray):
        res_fun = x
    else:
        res_fun = GridFunction(A.domain, coefficients=x.ravel())

    if return_residuals and return_iteration_count:
        return res_fun, info, callback.residuals, callback.count

    if return_residuals:
        return res_fun, info, callback.residuals

    if return_iteration_count:
        return res_fun, info, callback.count

    return res_fun, info


def _gmres_block_op_imp(
    A,
    b,
    tol=1e-5,
    restart=None,
    maxiter=None,
    use_strong_form=False,
    return_residuals=False,
    return_iteration_count=False,
):
    """Run implementation of GMRES for blocked operators."""
    import scipy.sparse.linalg

    import bempp.api
    import time
    from bempp.api.assembly.blocked_operator import (
        coefficients_from_grid_functions_list,
        projections_from_grid_functions_list,
        grid_function_list_from_coefficients,
    )

    # Assemble weak form before the logging messages

    if use_strong_form:
        b_vec = coefficients_from_grid_functions_list(b)
        A_op = A.strong_form()
    else:
        A_op = A.weak_form()
        b_vec = projections_from_grid_functions_list(b, A.dual_to_range_spaces)

    callback = IterationCounter(return_residuals)

    bempp.api.log("Starting GMRES iteration")
    start_time = time.time()
    x, info = scipy.sparse.linalg.gmres(
        A_op, b_vec, tol=tol, restart=restart, maxiter=maxiter, callback=callback
    )
    end_time = time.time()
    bempp.api.log(
        "GMRES finished in %i iterations and took %.2E sec."
        % (callback.count, end_time - start_time)
    )

    res_fun = grid_function_list_from_coefficients(x.ravel(), A.domain_spaces)

    if return_residuals and return_iteration_count:
        return res_fun, info, callback.residuals, callback.count

    if return_residuals:
        return res_fun, info, callback.residuals

    if return_iteration_count:
        return res_fun, info, callback.count

    return res_fun, info

In [5]:
precision = 5

h = 2 * np.pi/(precision*k0)
grid = bempp.api.shapes.sphere(h=h)


print(h, ': h')
print(precision, ': precision')
print(grid.number_of_edges * 2, ': N')

    
dA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

p1dA = [bempp.api.function_space(grid, "DP", 1, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

rA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
tA = [bempp.api.function_space(grid, "SNC", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

multitrace_ops = []
osrc_ops = []

# > Assemble all diagonal operators
for index in range(M):
  k = k_list[index]
  mu = mu_list[index]
  eta = eta_rel_list[index]
  efie = bempp.api.operators.boundary.maxwell.electric_field(dA[1], rA[1], tA[1], k, assembler='fmm')
  osrc = osrc_MtE(dA[1], rA[1], tA[1], p1dA[1], k)
  mfie = bempp.api.operators.boundary.maxwell.magnetic_field(dA[1], rA[1], tA[1], k, assembler='fmm')
  block_osrc = bempp.api.BlockedOperator(2,2)
  block_osrc[0,1] = eta * osrc
  block_osrc[1,0] = -1/eta * osrc
  osrc_ops.append(block_osrc)
  multitrace_ops.append(bempp.api.GeneralizedBlockedOperator([[mfie, eta * efie],[- 1/eta * efie, mfie]]))
  #osrc_ops.append(bempp.api.GeneralizedBlockedOperator([[zero, eta * osrc],[- 1/eta * osrc, zero]]))

lhs_op = multitrace_ops[0] + multitrace_ops[1]    

prec_op = osrc_ops[0] + osrc_ops[1]    

rhs = [bempp.api.GridFunction(rA[1], dual_space = tA[1], fun=tangential_trace),
      bempp.api.GridFunction(rA[1], dual_space = tA[1], fun=neumann_trace)]

b = bempp.api.assembly.blocked_operator.projections_from_grid_functions_list(rhs, lhs_op.dual_to_range_spaces)

bempp:HOST:INFO: Created grid with id 27cff036-a845-4f77-9653-f1cae561d463. Elements: 628. Edges: 942. Vertices: 316
bempp:HOST:WARNING: Setting include_boundary_dofs has no effect on this space type.
bempp:HOST:WARNING: Setting include_boundary_dofs has no effect on this space type.


0.25132741228718347 : h
5 : precision
1884 : N


/usr/local/lib/python3.10/dist-packages/bempp/api/assembly/discrete_boundary_operator.py:639: SparseEfficiencyWarning: splu converted its input to CSC format
  solver = solver_interface(actual_mat)
/usr/local/lib/python3.10/dist-packages/bempp/api/assembly/discrete_boundary_operator.py:639: SparseEfficiencyWarning: splu converted its input to CSC format
  solver = solver_interface(actual_mat)


In [6]:
#from scipy.sparse.linalg import gmres

P = prec_op.weak_form()
op_wf = lhs_op.weak_form()

x_gmres, conv_gmres, res_gmres = gmres(P * op_wf, P * b, return_residuals=True, restart = 1000, tol=1e-4)

bempp:HOST:INFO: OpenCL CPU Device set to: pthread-AMD EPYC 7302 16-Core Processor
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:499: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, devices)
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:499: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, devices)
bempp

In [7]:
sol = bempp.api.assembly.blocked_operator.grid_function_list_from_coefficients(x_gmres.ravel(), lhs_op.domain_spaces)

In [8]:
far_field_points = config['far_field_points']
electric_far = bempp.api.operators.far_field.maxwell.electric_field(sol[1].space, far_field_points, k0)
magnetic_far = bempp.api.operators.far_field.maxwell.magnetic_field(sol[0].space, far_field_points, k0)    
far_field = - electric_far * sol[1] - magnetic_far * sol[0]

A22 = far_field[2,:]
uh = 10 * np.log10(4 * np.pi * np.abs(A22[:1801]))
u =  10 * np.log10(4 * np.pi * np.abs(s1 / (-1j * k_ext) ))
rel_error = np.linalg.norm(uh - u) / np.linalg.norm(u)

print(rel_error)

66 warnings generated.
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:417: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, [devices[i] for i in to_be_built_indices])
66 warnings generated.


0.011116520650263287
